# Fitting crescita COVID 19

In [60]:
#Importazione librerie operative
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from scipy.optimize import curve_fit
from ipywidgets import interact

In [61]:
#Lettura dati sperimentali via web
df = pd.read_csv('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv')

### Impostazione funzione di fitting - curva "logistica"
  
### $$ N(N_{tot},N_0,r,t) = \frac{N_{tot}N_0}{N_0+(N_{tot}-N_0)e^{-rt}}$$


In [69]:
def sigma(x,a,b,c):
    y=(a*b)/(b+(a-b)*np.exp(-c*x))
    return y

In [70]:
Nexp = df['totale_casi']
a = df.count()
b = np.arange(a['data'])
t = b+1
t1= np.arange(60)
t2= t1-1
Prog = []
i = 0
for i in range(a['data']-1):
    ii = i+1
    pro = Nexp[ii]-Nexp[i]
    Prog.append(pro)

In [72]:
df_sel=df.drop(['data','stato','note_it', 'note_en'],axis=1)
@interact(parametro=df_sel.columns)
def sel(parametro):
    fg=go.Figure()
    fg.add_trace(go.Scatter(x=df.data ,y=df[parametro], mode='markers', marker_color = 'green'))
    fg.show()

interactive(children=(Dropdown(description='parametro', options=('ricoverati_con_sintomi', 'terapia_intensiva'…

##### Fit dati sperimentali

In [65]:
popt, pcov = curve_fit(sigma, t, Nexp, maxfev=5000, method='trf', bounds=(0, [200000., 300., 0.5]))
fit = sigma(t, *popt)
k, N0, r = popt
print('Ntotale=' ,int(popt[0])  ,'N0=' ,int(popt[1])  ,'r=' ,"%0.2f" % popt[2])
der = sigma(t1,*popt) - sigma(t2,*popt)
popt

Ntotale= 146212 N0= 299 r= 0.20


array([1.46212028e+05, 3.00000000e+02, 2.00970917e-01])

### Plot risultati ottenuti

In [66]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=t, y=Nexp, mode='markers',name='n°casi totali', marker=dict(symbol=[200],color='red', size=8)))
fig.add_trace(go.Scatter(x=t1, y=sigma(t1,*popt), mode='lines',name='Previsione',marker_color='green'))
fig.update_layout(title='Crescita casi COVID 19', \
                  xaxis_title='giorni da 24 feb',yaxis_title='n° totale casi')

In [67]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=t, y=Prog, mode='markers',name='incremento casi/giorno', \
                         marker=dict(symbol=[200],color='red', size=10)))
fig.add_trace(go.Scatter(x=t1, y=der, mode='lines',name='Previsione',marker_color='blue'))
fig.update_layout(title='Incremento giornaliero COVID 19', \
                  xaxis_title='giorni da 24 feb',yaxis_title='n° casi giornalieri')

In [75]:
tab=df.drop(['stato','note_it', 'note_en'],axis=1)
tab

,data,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,totale_casi,tamponi
0,2020-02-24T18:00:00,101,26,127,94,221,0,221,1,7,229,4324
1,2020-02-25T18:00:00,114,35,150,162,311,90,93,1,10,322,8623
2,2020-02-26T18:00:00,128,36,164,221,385,74,78,3,12,400,9587
3,2020-02-27T18:00:00,248,56,304,284,588,203,250,45,17,650,12014
4,2020-02-28T18:00:00,345,64,409,412,821,233,238,46,21,888,15695
5,2020-02-29T18:00:00,401,105,506,543,1049,228,240,50,29,1128,18661
6,2020-03-01T18:00:00,639,140,779,798,1577,528,566,83,34,1694,21127
7,2020-03-02T18:00:00,742,166,908,927,1835,258,342,149,52,2036,23345
8,2020-03-03T18:00:00,1034,229,1263,1000,2263,428,466,160,79,2502,25856
9,2020-03-04T18:00:00,1346,295,1641,1065,2706,443,587,276,107,3089,29837
